## Face detection

La ProCam s.p.a ha intenzione di lanciare sul mercato una nuova fotocamera digitale compatta ed economica destinata a piccoli fotografi in erba.

Vieni assunto come Data Scientist per realizzare il sistema di identificazione dei volti nelle immagini, questo permetterà poi ai tecnici della fotografia di ottimizzare le impostazioni per un selfie con una o più persone.

Si tratta di un problema di computer vision, più precisamente di Face Detection.

Devi fornire una pipeline scikit-learn che prende un'immagine in ingresso e ritorna una lista con le coordinate dei bounding box dove sono presenti dei volti, se nell'immagine non contiene volti la lista sarà ovviamente vuota.

- Non ti viene fornito un dataset, sta a te cercarne uno in rete o, nella peggiore delle ipotesi, costruirlo, per semplicità non considereremo implicazioni sulle licenze ad utilizzo commerciale, si tratta pur sempre di un progetto didattico.
- Non puoi utilizzare modelli pre-addestrati, devi addestrarlo tu utilizzando scikit-learn.
- Stai lavorando su un sistema con ridotte capacità di calcolo, quindi il modello deve richiedere poche risorse di calcolo.
- Ovviamente non ti vengono fornite indicazioni sull'implementazione, fai un'approfondita ricerca bibliografica per trovare la soluzione migliore da adottare, il notebook che consegnerai deve essere ben documentato, devi spiegare quali soluzioni hai adottato e perché ed ogni risorsa esterna (paper, blog post, codice github...) che hai utilizzato.
- Il progetto è abbastanza complesso, ricorda che in caso ne avessi necessità puoi sempre chiedere aiuto ai tuoi coach nella Classe Virtuale di Machine Learning su Discord.

https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/

https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html

In [8]:
#Import some modules
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.transform import resize
from skimage.io import imread
from skimage.exposure import rescale_intensity
from sklearn.datasets import fetch_lfw_people
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


Prima di tutto provo a leggere le immagini di esempio di sklearn e provo ad estrarre hog features da questo... Il resto dopo

In [2]:
#impoort first dataset (faces)
lfw_people = fetch_lfw_people(resize=1)

images_positive = lfw_people.images
#use only a portion of these images, to don't unbalance final dataset
# indexes = np.random.choice(len(lfw_people.images),1600, replace=False)
# images_positive = lfw_people.images[indexes]

#some helper variables
#size = 64 x 128 as original paper
resize_shape = (128, 64) #as row x columns (h x w)

#init list to hold features arrays and labels
features_list = []
labels_list = []

In [4]:
X_positive = images_positive
y_positive = np.ones(X_positive.shape[0])
print(X_positive.shape) #3 dimensions: 1 = records, 2-3 = image as matrix 
print(y_positive.shape)

(13233, 125, 94)
(13233,)


In [6]:
print(X_positive[1,:,:].shape)

(125, 94)


In [7]:
#process negative images
negative_images = []

butterflies_img_dir = 'leedsbutterfly/images'
background_img_dir = 'stanford-background-dataset/images'
cat_img_dir = 'animals/raw-img/gatto'
chicken_img_dir = 'animals/raw-img/gallina'
cow_img_dir = 'animals/raw-img/mucca'
squirrel_img_dir = 'animals/raw-img/scoiattolo'
sheep_img_dir = 'animals/raw-img/pecora'
negative_img_dirs = [butterflies_img_dir,
                     background_img_dir,
                     cat_img_dir,
                     chicken_img_dir,
                     cow_img_dir,
                     squirrel_img_dir,
                     sheep_img_dir                    
                    ]

# for directory in tqdm(negative_img_dirs,desc="Dataset:",unit="item"):
for directory in negative_img_dirs:
    for filename in tqdm(os.listdir(directory),desc=f"Processing negative images ({directory})",
                        unit="item"):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
            img_path = os.path.join(directory, filename)
            img = imread(img_path, as_gray=True)
            #this resize is necessary for concatenation in a single numpy array
            img = resize(img, X_positive[1,:,:].shape)
            negative_images.append(img)

X_negative = np.array(negative_images)
y_negative = np.zeros(X_negative.shape[0])
print(X_negative.shape) #3 dimensions: 1 = records, 2-3 = image as matrix 
print(y_negative.shape)

Processing negative images (leedsbutterfly/images): 100%|██████████| 832/832 [00:24<00:00, 33.81item/s]
Processing negative images (stanford-background-dataset/images): 100%|██████████| 715/715 [00:03<00:00, 235.88item/s]
Processing negative images (animals/raw-img/gatto): 100%|██████████| 1668/1668 [01:28<00:00, 18.90item/s] 
Processing negative images (animals/raw-img/gallina): 100%|██████████| 3098/3098 [00:15<00:00, 199.77item/s]
Processing negative images (animals/raw-img/mucca): 100%|██████████| 1866/1866 [00:09<00:00, 190.96item/s]
Processing negative images (animals/raw-img/scoiattolo): 100%|██████████| 1862/1862 [00:10<00:00, 172.73item/s]
Processing negative images (animals/raw-img/pecora): 100%|██████████| 1820/1820 [00:16<00:00, 112.01item/s]


(11861, 125, 94)
(11861,)


In [13]:
X = np.vstack((X_positive, X_negative))
y = np.concatenate([y_positive,y_negative])

print(X.shape)
print(y.shape)

(25094, 125, 94)
(25094,)


Primo tentativo di pipeline...

In [17]:
from ImageResizer import ImageResizer
from HOGFeatureExtractor import HOGFeatureExtractor

resize_shape = (128, 64) #as row x columns (h x w)

# Definisci la pipeline
pipeline = Pipeline([
    ('resizer', ImageResizer(resize_shape)),
    ('hog', HOGFeatureExtractor()),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=100)),  # Scegli il numero di componenti che desideri
    ('svc', SVC(kernel='poly'))  # SVC con kernel polinomiale
])

# Addestra il modello
pipeline.fit(X, y)



HOG feature extraction: 100%|██████████| 25094/25094 [01:10<00:00, 354.74item/s]


Pipeline(steps=[('resizer', ImageResizer()), ('hog', HOGFeatureExtractor()),
                ('scaler', StandardScaler()), ('pca', PCA(n_components=100)),
                ('svc', SVC(kernel='poly'))])

In [21]:

print(pipeline.predict([X[-100,:,:]]))

HOG feature extraction: 100%|██████████| 1/1 [00:00<00:00, 165.59item/s]

[0.]


Import object images from kaggle

In [ ]:
# !pip install kaggle

Butterfly dataset (832 images). 

It contains bufferflies images with some flowers and plants. With this dataset model can learn to detect naturalistic objects.

In [ ]:
!kaggle datasets download veeralakrishna/butterfly-dataset

Dataset URL: https://www.kaggle.com/datasets/veeralakrishna/butterfly-dataset
License(s): Community Data License Agreement - Permissive - Version 1.0
butterfly-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# !unzip butterfly-dataset.zip -d butterfly-dataset
!tar -xf butterfly-dataset.zip

Background dataset (715 images).

It contains some backgournd images, taken from streets and landscape photos. With this dataset model can learn to detect common background objects and patterns.

In [ ]:
!mkdir stanford-background-dataset

Sottodirectory o file stanford-background-dataset già esistente.


In [ ]:
!kaggle datasets download -p stanford-background-dataset balraj98/stanford-background-dataset 

Dataset URL: https://www.kaggle.com/datasets/balraj98/stanford-background-dataset
License(s): other
stanford-background-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# !unzip stanford-background-dataset.zip -d stanford-background-dataset
!tar -xf stanford-background-dataset/stanford-background-dataset.zip -C stanford-background-dataset

Add some animals pcitures :)

In [ ]:
!mkdir animals

Sottodirectory o file animals già esistente.


In [ ]:
!kaggle datasets download -p animals alessiocorrado99/animals10

Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
License(s): GPL-2.0
animals10.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!tar -xf animals/animals10.zip -C animals

# TODO: Inserire il random searchCV con la pipeline custom creata sopra

https://scikit-learn.org/stable/modules/compose.html

https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html#sphx-glr-auto-examples-compose-plot-digits-pipe-py

https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html#sphx-glr-auto-examples-compose-plot-compare-reduction-py